In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import toolkitETL
import importlib
importlib.reload(toolkitETL)

<module 'toolkitETL' from 'c:\\Users\\pedro\\OneDrive\\Área de Trabalho\\Pedro\\Portifolio\\ETL-ML-Dengue\\src\\etl\\toolkitETL.py'>

In [2]:
## Estabelecendo diretórios base
diretorio_src_etl = os.getcwd()
diretorio_src = Path(diretorio_src_etl).parent.absolute()
diretorio_base = Path(diretorio_src).parent.absolute()
diretorio_base = "D:\\Mestrado_pedro"
diretorio_data = os.path.join(diretorio_base, 'data')

# Bronze
diretorio_bronze = os.path.join(diretorio_data, 'bronze')
diretorio_cnes_st_bronze = os.path.join(diretorio_bronze, 'cnes_st')
diretorio_cnes_st_bronze_csv = os.path.join(diretorio_cnes_st_bronze, 'csv')

# Silver
diretorio_silver = os.path.join(diretorio_data, 'silver')
diretorio_cnes_st_silver = os.path.join(diretorio_silver, 'cnes_st')
diretorio_cnes_st_silver_tsv = os.path.join(diretorio_cnes_st_silver, 'tsv')

## Criação de pastas para armazenar os arquivos
os.makedirs(os.path.join(diretorio_data), exist_ok=True)
os.makedirs(os.path.join(diretorio_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_st_silver), exist_ok=True)
os.makedirs(os.path.join(diretorio_cnes_st_silver_tsv), exist_ok=True)




In [3]:
arquivos_csv = [arquivo for arquivo in os.listdir(diretorio_cnes_st_bronze_csv) if arquivo.endswith("csv")]
arquivos_csv = arquivos_csv[-5:]
arquivos_tsv = [arquivo.rstrip("csv") + "tsv" for arquivo in arquivos_csv]
arquivos_tsv

['2016_1_2016_12_PE_CNES-ST.tsv',
 '2016_1_2016_12_RN_CNES-ST.tsv',
 '2016_1_2016_12_RO_CNES-ST.tsv',
 '2016_1_2016_12_RR_CNES-ST.tsv',
 '2016_1_2016_12_SP_CNES-ST.tsv']

In [11]:
colunas_importantes = ['CNES', 'CODUFMUN', 'COMPETEN', 'VINC_SUS', 'PF_PJ', 'NIV_DEP', 'TPGESTAO', 'ORGEXPED', 'AV_ACRED', 'CLASAVAL', 'AV_PNASS', 'GESPRG1E', 'GESPRG1M', 'GESPRG2E', 'GESPRG2M', 'GESPRG4E', 'GESPRG4M', 'NIVATE_A', 'GESPRG3E', 'GESPRG3M', 'GESPRG5E', 'GESPRG5M', 'GESPRG6E', 'GESPRG6M', 'NIVATE_H', 'QTLEITP1', 'QTLEITP2', 'QTLEITP3', 'LEITHOSP', 'QTINST01', 'QTINST02', 'QTINST03', 'QTINST04', 'QTINST05', 'QTINST06', 'QTINST07', 'QTINST08', 'QTINST09', 'QTINST10', 'QTINST11', 'QTINST12', 'QTINST13', 'QTINST14', 'URGEMERG', 'QTINST15', 'QTINST16', 'QTINST17', 'QTINST18', 'QTINST19', 'QTINST20', 'QTINST21', 'QTINST22', 'QTINST23', 'QTINST24', 'QTINST25', 'QTINST26', 'QTINST27', 'QTINST28', 'QTINST29', 'QTINST30', 'ATENDAMB', 'QTINST31', 'QTINST32', 'QTINST33', 'CENTRCIR', 'QTINST34', 'QTINST35', 'QTINST36', 'QTINST37', 'CENTROBS', 'QTLEIT05', 'QTLEIT06', 'QTLEIT07', 'QTLEIT08', 'QTLEIT09', 'QTLEIT19', 'QTLEIT20', 'QTLEIT21', 'QTLEIT22', 'QTLEIT23', 'QTLEIT32', 'QTLEIT34', 'QTLEIT38', 'QTLEIT39', 'QTLEIT40', 'CENTRNEO', 'ATENDHOS', 'SERAP01P', 'SERAP01T', 'SERAP02P', 'SERAP02T', 'SERAP03P', 'SERAP03T', 'SERAP04P', 'SERAP04T', 'SERAP05P', 'SERAP05T', 'SERAP06P', 'SERAP06T', 'SERAP07P', 'SERAP07T', 'SERAP08P', 'SERAP08T', 'SERAP09P', 'SERAP09T', 'SERAP10P', 'SERAP10T', 'SERAP11P', 'SERAP11T', 'SERAPOIO', 'RES_BIOL', 'RES_QUIM', 'RES_RADI', 'RES_COMU', 'COLETRES', 'COMISS01', 'COMISS02', 'COMISS03', 'COMISS04', 'COMISS05', 'COMISS06', 'COMISS07', 'COMISS08', 'COMISS09', 'COMISS10', 'COMISS11', 'COMISS12', 'COMISSAO', 'AP01CV01', 'AP01CV02', 'AP01CV03', 'AP01CV04', 'AP01CV05', 'AP01CV06', 'AP02CV01', 'AP02CV02', 'AP02CV03', 'AP02CV04', 'AP02CV05', 'AP02CV06', 'AP03CV01', 'AP03CV02', 'AP03CV03', 'AP03CV04', 'AP03CV05', 'AP03CV06', 'AP04CV01', 'AP04CV02', 'AP04CV03', 'AP04CV04', 'AP04CV05', 'AP04CV06', 'AP05CV01', 'AP05CV02', 'AP05CV03', 'AP05CV04', 'AP05CV05', 'AP05CV06', 'AP06CV01', 'AP06CV02', 'AP06CV03', 'AP06CV04', 'AP06CV05', 'AP06CV06', 'AP07CV01', 'AP07CV02', 'AP07CV03', 'AP07CV04', 'AP07CV05', 'AP07CV06', 'ATEND_PR', 'COD_IR', 'ESFERA_A', 'RETENCAO', 'ATIVIDAD', 'NATUREZA', 'CLIENTEL', 'TURNO_AT', 'NIV_HIER', 'TP_PREST']

colunas_decodificar = {
    "PF_PJ": {
        1: "Física",
        3: "Jurídica"
    },
    "NIV_DEP": {
        1: "Individual",
        3: "Mantida"
    },
    "COD_IR": {
        10: "Estabelecimento PUBLICO",
        11: "Estabelecimento FILANTROPICO",
        12: "Estabelecimento SEM FINS LUCRATIVOS",
        13: "Estabelecimento PRIVADO LUCRATIVA SIMPLES",
        14: "Estabelecimento PRIVADO LUCRATIVA",
        15: "Estabelecimento SINDICAL",
        16: "Estabelecimento PESSOA FISICA",
        19: "Estabelecimento Ret.Manten.código 19",
        "IR": "Estabelecimento Ret.Manten.código IR"
    },
    "TPGESTAO": {
        "Z": "Não informado",
        "D": "Dupla",
        "E": "Estadual",
        "M": "Municipal",
        "S": "Sem gestão"
    },
    "ESFERA_A": {
        1: "Federal",
        2: "Estadual",
        3: "Municipal",
        4: "Privada",
        99: "Esfera não informada"
    },
    "RETENCAO": {
        0: "Retenção estab. não informada",
        99: "Retenção estab. não informada",
        10: "Estabelecimento PUBLICO",
        11: "Estabelecimento FILANTROPICO",
        12: "Estabelecimento SEM FINS LUCRATIVOS",
        13: "Estabelecimento PRIVADO LUCRATIVA SIMPLES",
        14: "Estabelecimento PRIVADO LUCRATIVA",
        15: "Estabelecimento SINDICAL",
        16: "Estabelecimento PESSOA FISICA"
    },
    "ATIVIDAD": {
        1: "Unidade Universitária",
        2: "Unidade Escola Superior Isolada",
        3: "Unidade Auxiliar de Ensino",
        4: "Unidade SEM atividade de Ensino",
        5: "Hospital de Ensino",
        99: "Atividade Ensino não informada"
    },
    "NATUREZA": {
        1: "Administração Direta da Saúde (MS, SES, e SMS)",
        2: "Adm Direta outros orgãos (MEX, MEx, Marinha,...)",
        3: "Adm Indireta - Autarquias",
        4: "Adm Indireta - Fundação Pública",
        5: "Adm Indireta - Empresa Pública",
        6: "Adm Indireta - Organização Social Pública",
        7: "Empresa Privada",
        8: "Fundação Privada",
        9: "Cooperativa",
        10: "Serviço Social Autônomo",
        11: "Entidade Beneficente SEM fins lucrativos",
        12: "Economia Mista",
        13: "Sindicato",
        0: "Natureza inexistente",
        99: "Natureza não informada"
    },
    "CLIENTEL": {
        1: "Atendimento de demanda espontânea",
        2: "Atendimento de demanda referenciada",
        3: "Atendimento de demanda espontânea e referenciada",
        0: "Fluxo de Clientela não exigido",
        99: "Fluxo de Clientela não informado"
    },
    "TURNO_AT": {
        1: "ATENDIMENTO TURNOS INTERMITENTES",
        2: "ATENDIMENTO CONTÍNUO 24 HORAS/DIA (Pl Sab Dom Fer)",
        3: "ATENDIMENTO TURNOS MANHÃ/TARDE/NOITE",
        4: "ATENDIMENTO SOMENTE PELA MANHÃ",
        5: "ATENDIMENTO SOMENTE À TARDE",
        6: "ATENDIMENTO TURNOS MANHÃ/TARDE",
        7: "ATENDIMENTO SOMENTE À NOITE",
        99: "Turno não informado"
    },
    "NIV_HIER": {
        1: "NH 1-PAB-PABA",
        2: "NH 2-Média M1",
        3: "NH 3-Média M2 e M3",
        4: "NH 4-AltaComplex.Ambul.",
        5: "NH 5-Baixa M1 e M2",
        6: "NH 6-Média M2 e M3",
        7: "NH 7-Média M3",
        8: "NH 8-AltaCompl.Hos/Amb.",
        0: "NH não informado",
        99: "NH não informado"
    },
    "TP_PREST": {
        30: "PUBLICO FEDERAL",
        40: "PUBLICO ESTADUAL",
        50: "PUBLICO MUNICIPAL",
        61: "FILANTROPICO COM CNAS VALIDO",
        80: "SINDICATO",
        20: "PRIVADO COM FINS LUCRATIVOS",
        22: "PRIVADO OPTANTE PELO SIMPLES",
        60: "PRIVADO SEM FINS LUCRATIVOS",
        99: "TIPO DE PRESTADOR NÃO INFORMADO"
    },
    "ORGEXPED": {
        1: "SES",
        2: "SMS"
    },
    "CLASAVAL": {
        1: "ACREDITADO NO NÍV.1",
        2: "ACREDITADO NO NÍV.2",
        3: "ACREDITADO NO NÍV.3",
        0: "NÃO ATENDEU AOS PADRÕES MÍNIMOS",
        9: "Nível Avaliação não informado"
    }, ### A partir daqui são apenas colunas varchar(3)
    "VINC_SUS": {1: "Sim", 2: "Não"},
    "AV_ACRED": {1: "Sim", 2: "Não"},
    "AV_PNASS": {1: "Sim", 2: "Não"},
    "GESPRG1E": {1: "Sim", 2: "Não"},
    "GESPRG1M": {1: "Sim", 2: "Não"},
    "GESPRG2E": {1: "Sim", 2: "Não"},
    "GESPRG2M": {1: "Sim", 2: "Não"},
    "GESPRG4E": {1: "Sim", 2: "Não"},
    "GESPRG4M": {1: "Sim", 2: "Não"},
    "NIVATE_A": {1: "Sim", 2: "Não"},
    "GESPRG3E": {1: "Sim", 2: "Não"},
    "GESPRG3M": {1: "Sim", 2: "Não"},
    "GESPRG5E": {1: "Sim", 2: "Não"},
    "GESPRG5M": {1: "Sim", 2: "Não"},
    "GESPRG6E": {1: "Sim", 2: "Não"},
    "GESPRG6M": {1: "Sim", 2: "Não"},
    "NIVATE_H": {1: "Sim", 2: "Não"},
    "LEITHOSP": {1: "Sim", 2: "Não"},
    "URGEMERG": {1: "Sim", 2: "Não"},
    "ATENDAMB": {1: "Sim", 2: "Não"},
    "CENTRCIR": {1: "Sim", 2: "Não"},
    "CENTROBS": {1: "Sim", 2: "Não"},
    "CENTRNEO": {1: "Sim", 2: "Não"},
    "ATENDHOS": {1: "Sim", 2: "Não"},
    "SERAP01P": {1: "Sim", 2: "Não"},
    "SERAP01T": {1: "Sim", 2: "Não"},
    "SERAP02P": {1: "Sim", 2: "Não"},
    "SERAP02T": {1: "Sim", 2: "Não"},
    "SERAP03P": {1: "Sim", 2: "Não"},
    "SERAP03T": {1: "Sim", 2: "Não"},
    "SERAP04P": {1: "Sim", 2: "Não"},
    "SERAP04T": {1: "Sim", 2: "Não"},
    "SERAP05P": {1: "Sim", 2: "Não"},
    "SERAP05T": {1: "Sim", 2: "Não"},
    "SERAP06P": {1: "Sim", 2: "Não"},
    "SERAP06T": {1: "Sim", 2: "Não"},
    "SERAP07P": {1: "Sim", 2: "Não"},
    "SERAP07T": {1: "Sim", 2: "Não"},
    "SERAP08P": {1: "Sim", 2: "Não"},
    "SERAP08T": {1: "Sim", 2: "Não"},
    "SERAP09P": {1: "Sim", 2: "Não"},
    "SERAP09T": {1: "Sim", 2: "Não"},
    "SERAP10P": {1: "Sim", 2: "Não"},
    "SERAP10T": {1: "Sim", 2: "Não"},
    "SERAP11P": {1: "Sim", 2: "Não"},
    "SERAP11T": {1: "Sim", 2: "Não"},
    "SERAPOIO": {1: "Sim", 2: "Não"},
    "RES_BIOL": {1: "Sim", 2: "Não"},
    "RES_QUIM": {1: "Sim", 2: "Não"},
    "RES_RADI": {1: "Sim", 2: "Não"},
    "RES_COMU": {1: "Sim", 2: "Não"},
    "COLETRES": {1: "Sim", 2: "Não"},
    "COMISS01": {1: "Sim", 2: "Não"},
    "COMISS02": {1: "Sim", 2: "Não"},
    "COMISS03": {1: "Sim", 2: "Não"},
    "COMISS04": {1: "Sim", 2: "Não"},
    "COMISS05": {1: "Sim", 2: "Não"},
    "COMISS06": {1: "Sim", 2: "Não"},
    "COMISS07": {1: "Sim", 2: "Não"},
    "COMISS08": {1: "Sim", 2: "Não"},
    "COMISS09": {1: "Sim", 2: "Não"},
    "COMISS10": {1: "Sim", 2: "Não"},
    "COMISS11": {1: "Sim", 2: "Não"},
    "COMISS12": {1: "Sim", 2: "Não"},
    "COMISSAO": {1: "Sim", 2: "Não"},
    "AP01CV01": {1: "Sim", 2: "Não"},
    "AP01CV02": {1: "Sim", 2: "Não"},
    "AP01CV05": {1: "Sim", 2: "Não"},
    "AP01CV06": {1: "Sim", 2: "Não"},
    "AP01CV03": {1: "Sim", 2: "Não"},
    "AP01CV04": {1: "Sim", 2: "Não"},
    "AP02CV01": {1: "Sim", 2: "Não"},
    "AP02CV02": {1: "Sim", 2: "Não"},
    "AP02CV05": {1: "Sim", 2: "Não"},
    "AP02CV06": {1: "Sim", 2: "Não"},
    "AP02CV03": {1: "Sim", 2: "Não"},
    "AP02CV04": {1: "Sim", 2: "Não"},
    "AP03CV01": {1: "Sim", 2: "Não"},
    "AP03CV02": {1: "Sim", 2: "Não"},
    "AP03CV05": {1: "Sim", 2: "Não"},
    "AP03CV06": {1: "Sim", 2: "Não"},
    "AP03CV03": {1: "Sim", 2: "Não"},
    "AP03CV04": {1: "Sim", 2: "Não"},
    "AP04CV01": {1: "Sim", 2: "Não"},
    "AP04CV02": {1: "Sim", 2: "Não"},
    "AP04CV05": {1: "Sim", 2: "Não"},
    "AP04CV06": {1: "Sim", 2: "Não"},
    "AP04CV03": {1: "Sim", 2: "Não"},
    "AP04CV04": {1: "Sim", 2: "Não"},
    "AP05CV01": {1: "Sim", 2: "Não"},
    "AP05CV02": {1: "Sim", 2: "Não"},
    "AP05CV05": {1: "Sim", 2: "Não"},
    "AP05CV06": {1: "Sim", 2: "Não"},
    "AP05CV03": {1: "Sim", 2: "Não"},
    "AP05CV04": {1: "Sim", 2: "Não"},
    "AP06CV01": {1: "Sim", 2: "Não"},
    "AP06CV02": {1: "Sim", 2: "Não"},
    "AP06CV05": {1: "Sim", 2: "Não"},
    "AP06CV06": {1: "Sim", 2: "Não"},
    "AP06CV03": {1: "Sim", 2: "Não"},
    "AP06CV04": {1: "Sim", 2: "Não"},
    "AP07CV01": {1: "Sim", 2: "Não"},
    "AP07CV02": {1: "Sim", 2: "Não"},
    "AP07CV05": {1: "Sim", 2: "Não"},
    "AP07CV06": {1: "Sim", 2: "Não"},
    "AP07CV03": {1: "Sim", 2: "Não"},
    "AP07CV04": {1: "Sim", 2: "Não"},
    "ATEND_PR": {1: "Sim", 2: "Não"},
}

column_map = []

columns_to_convert = []

[]


In [8]:
for arquivo_csv, arquivo_tsv in zip(arquivos_csv, arquivos_tsv):
    df = pd.read_csv(os.path.join(diretorio_cnes_st_bronze_csv, arquivo_csv), sep=',')

    for coluna in colunas_importantes:
        if coluna not in df.columns:
            df[coluna] = np.nan
            continue
    df = df[colunas_importantes]

    for coluna in colunas_decodificar:
        df[coluna] = df[coluna].apply(lambda x: toolkitETL.aplicar_decoficador(x, coluna, colunas_decodificar))
    df.replace("NA", np.nan, inplace=True)
    df.replace("", np.nan, inplace=True)
    df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
    df['NU_MES'] = df['COMPETEN'].apply(lambda x: str(x)[-2:].lstrip("0"))
    df['NU_ANO'] = df['COMPETEN'].apply(lambda x: str(x)[:4])
    # df = df.rename(columns=column_map)
    
    df.columns = df.columns.str.lower()
    
    # for column in columns_to_convert:
    #     df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

    df.to_csv(os.path.join(diretorio_cnes_st_silver_tsv, arquivo_tsv), sep='\t', index=False)

In [14]:
diretorio_dotenv = os.path.join(diretorio_base, ".env")

cursor, conn = toolkitETL.conectar_banco(diretorio_dotenv)
for arquivo_tsv in arquivos_tsv:
    
    caminho_arquivo = os.path.join(diretorio_cnes_st_silver_tsv, arquivo_tsv)
    toolkitETL.inserir_no_banco(cursor, conn, caminho_arquivo, 'cnes_st', delimiter='\t',
                                        diretorio_dotenv=diretorio_dotenv)
